In [1]:
import numpy as np
import polars as pl
from tqdm import tqdm

from typing import List, Any

import scipy.sparse as sp
from sklearn.model_selection import train_test_split

import random
from collections import Counter

In [2]:
data = pl.read_parquet('train.parquet')
# датафрейм с обратными ребрами
data_rev = (
    data
    .rename({'uid': 'friend_uid', 'friend_uid': 'uid'})
    .select('uid', 'friend_uid')
)

# соединим все в один граф
data = pl.concat([data, data_rev])
data

uid,friend_uid
i64,i64
93464,114312
93464,103690
93464,108045
93464,116128
93464,94113
93464,101668
93464,118820
93464,93617
93464,97587


Данные состоят из двух колонок:

- `uid` – идентификатор пользователя
- `friend_uid` – идентификатор друга этого пользователя

Нашей задачей будет порекомендовать возможных друзей, для оценки вашего решения будет использоваться метрика Recall@10, равная проценту верно угаданных друзей

In [3]:
TOP_K = 10
RANDOM_STATE = 42

SUBMISSION_PATH = 'submission.parquet'


def user_intersection(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: number of items in intersection of y_rel and y_rec (truncated to top-K)
    """
    return len(set(y_rec[:k]).intersection(set(y_rel)))


def user_recall(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: percentage of found relevant items through recommendations
    """
    return user_intersection(y_rel, y_rec, k) / min(k, len(set(y_rel)))

## Валидация

Так как у нас нет временной последовательности и рекомендации друзей не так сильно зависят от временной составляющей, в качестве можно использовать случайно выбранные ребра в графе (при этом для каждого пользователя будет равная пропорция друзей в валидации, которую можно достичь с помощью stratify параметра)

In [4]:
# зафиксируем генератор случайных чисел
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [5]:
# отфильтруем тех пользователей, у которых только один друг :(
# для того, чтобы в тренировочной выборке и валидации было хотя бы по одному другу
friends_count = data.groupby('uid').count()
filtered_uid = set(friends_count.filter(pl.col('count') > 1)['uid'].to_list())

data_filtered = data.filter(pl.col('uid').is_in(filtered_uid))

# случайно выбираем ребра для валидационной выборки
train_df, test_df = train_test_split(
    data_filtered.filter(pl.col('uid').is_in(filtered_uid)),
    stratify=data_filtered['uid'],
    test_size=0.1,
    random_state=RANDOM_STATE
)

train_df

uid,friend_uid
i64,i64
62053,63575
31895,59356
97127,32271
89,11703
105178,47188
116127,52662
33824,15235
23690,103992
94660,45709


## Бейзлайн (Random)

In [6]:
grouped_df = (
    test_df
    .groupby('uid')
    .agg(pl.col('friend_uid').alias('y_rel'))
    .join(
        train_df
        .groupby('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)

median_seq_len = int(grouped_df['user_history'].apply(len).median())
print(f"среднее число uid в user_history: {median_seq_len}")

среднее число uid в user_history: 36


In [7]:
n_users = train_df['uid'].max() + 1

# количество друзей у каждого пользователя
friends_count = np.zeros(n_users)
for uid, count in Counter(train_df['uid']).items():
    friends_count[uid] = count
    
friends_count /= sum(friends_count)

In [8]:
recall_list = []
recs = np.random.choice(n_users, size=(n_users, TOP_K + median_seq_len), p=friends_count)

for user_id, y_rel, user_history in tqdm(grouped_df.rows()):
    y_rec = [uid for uid in recs[user_id] if uid not in user_history]
    recall_list.append(user_recall(y_rel, y_rec))
    
print(f'Recall@{TOP_K} = {np.mean(recall_list)}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 92562/92562 [00:04<00:00, 20068.17it/s]

Recall@10 = 0.0003110784946203369


## Построим рекомендации

In [9]:
# посчитаем вероятности уже по всем имеющимся данным
n_users = data['uid'].max() + 1

# количество друзей у каждого пользователя
friends_count = np.zeros(n_users)
for uid, count in Counter(data['uid']).items():
    friends_count[uid] = count
    
friends_count /= sum(friends_count)

In [10]:
sample_submission = pl.read_parquet('sample_submission.parquet')

grouped_df = (
    sample_submission.select('uid')
    .join(
        train_df
        .groupby('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)

submission = []
recs = np.random.choice(n_users, size=(n_users, TOP_K + median_seq_len), p=friends_count)

for user_id, user_history in tqdm(grouped_df.rows()):
    user_history = [] if user_history is None else user_history
    
    y_rec = [uid for uid in recs[user_id] if uid not in user_history]
    submission.append((user_id, y_rec))
    
submission = pl.DataFrame(submission, schema=['user_id', 'y_recs'])
submission.write_parquet('submission.parquet')
submission

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 85483/85483 [00:04<00:00, 18203.32it/s]


user_id,y_recs
i64,list[i64]
0,"[75174, 40482, … 107746]"
1,"[27663, 82181, … 37095]"
3,"[105454, 12906, … 43868]"
4,"[2627, 60169, … 108457]"
5,"[29164, 53357, … 33803]"
6,"[62015, 44506, … 3835]"
7,"[74067, 79534, … 53438]"
8,"[75913, 31803, … 32356]"
9,"[26079, 73565, … 111629]"
